
# YOLOV5 Baseline model

* Spot instance: i3.8xlarge
* Environement: pytorch_p36

# 1.Unzipping the data


See the Dataset_download notebook regarding the instrcution to download the dataset on an AWS server.


Choose which dataset you want to use and always unpack the labels, i.e. archive.

In [ ]:
!python --version

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Python 3.7.10


In [ ]:
!pip freeze

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
appdirs==1.4.4
argh==0.26.2
argon2-cffi @ file:///tmp/build/80754af9/argon2-cffi_1613036642480/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///tmp/build/80754af9/astroid_1613500851753/work
astropy @ file:///tmp/build/80754af9/astropy_1606922938720/work
async-generator==1.10
atomicwrites==1.4.0
attrs @ file:///tmp/build/80754af9/attrs_1604765588209/work
autopep8 @ file:///tmp/build/80754af9/autopep8_1613246362269/work
autovizwidget==0.18.0
Babel @ file:///tmp/build/80754af9/babel_1607110387436/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
bac

# 2.Cloning yolo and Installing and importing libraries

In [ ]:
import os
os.chdir('/home/ubuntu/GitHub')
!git clone https://github.com/ultralytics/yolov5 # clone repo
os.chdir('/home/ubuntu/GitHub/yolov5')
!pip install -qr requirements.txt
os.chdir('/home/ubuntu/GitHub')

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Cloning into 'yolov5'...
remote: Enumerating objects: 5437, done.
remote: Total 5437 (delta 0), reused 0 (delta 0), pack-reused 5437
Receiving objects: 100% (5437/5437), 8.07 MiB | 25.33 MiB/s, done.
Resolving deltas: 100% (3717/3717), done.
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


In [ ]:
!pip install wandb
#Ensamble boxes is needed for the weighted boxes fusion, WBF.
!pip install ensemble_boxes
#!pip3 install --upgrade Pillow

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
     |████████████████████████████████| 2.0 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 131 kB 22.6 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.5 MB/s  eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=79068d28909796dd74dbf5b948cef5f92b54eb104bde0e6dff8ba4d4f5fcda95
  Stored in directory: /home/ubuntu/.cache/pip/wheels/29/93/c6/762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=759e17906a4a2418fb2cd7f5697a44d58bcafd

In [ ]:
import pandas as pd
import os
import numpy as np
import shutil
import ast
from sklearn import model_selection
from tqdm import tqdm
import wandb
from sklearn.model_selection import GroupKFold, StratifiedKFold
from IPython.display import Image, clear_output  
from os import listdir
from os.path import isfile
from glob import glob
import yaml
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import cv2
from ensemble_boxes import *
from PIL import Image
from PIL import ImageFilter
from os import listdir
from os.path import isfile, join

Matplotlib is building the font cache; this may take a moment.


# 4.Preparing for YOLO and performing WBF.

In [ ]:
# Change link to Linus train csv file
train_df = pd.read_csv('/home/ubuntu/GitHub/Data/vinbigdata/train.csv')
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072


In [ ]:
train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

In [ ]:
# Change path to the train folder
train_df['image_path'] = f'/home/ubuntu/GitHub/Data/vinbigdata/train/'+train_df.image_id+'.png'
train_df.loc[0, 'image_path']

'/home/ubuntu/GitHub/Data/vinbigdata/train/9a5094b2563a1ef3ff50dc5c7ff71345.png'

WBF

We need not train our model on multiple annotations of the same abnormality. We shall use a technique called Weighted Boxes Fusion (WBF) to provide us with the best annotation. This will definitely reduce the metadata size by a lot. The algorithm utilizes confidence scores of all proposed bounding boxes to constructs the averaged boxes.

[Weighted Boxes Fusion: ensembling boxes for object detection models](https://arxiv.org/abs/1910.13302)

In [ ]:
#https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/208468

# ===============================
# Default WBF config 
iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1

mapping = {0:'Aortic enlargement', 4:'Consolidation',13:'Pulmonary fibrosis',3:'Cardiomegaly',
           7:'Lung Opacity', 6:'Infiltration',2:'Calcification',1:'Atelectasis',9:'Other lesion',12:'Pneumothorax',
           8:'Nodule/Mass',10:'Pleural effusion',5:'ILD',11:'Pleural thickening'}

# Loading the train DF

df = train_df
df.fillna(0, inplace=True)
df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

results = []
image_ids = df["image_id"].unique()

for image_id in tqdm(image_ids, total=len(image_ids)):
    #print(image_id)
    # All annotations for the current image.
    data = df[df["image_id"] == image_id]
    data = data.reset_index(drop=True)
    annotations = {}
    weights = []
    
    
    width=data.iloc[0].width
    height=data.iloc[0].height
    image_path=data.iloc[0].image_path
    #class_name=data.iloc[0].class_name

    # WBF expects the coordinates in 0-1 range.
    max_value = data.iloc[:, 4:8].values.max()
    data.loc[:, ["x_min", "y_min", "x_max", "y_max"]] = data.iloc[:, 4:8] / max_value

    # Loop through all of the annotations
    for idx, row in data.iterrows():
        #print(row)

        rad_id = row["rad_id"]
        
        #print(class_name)
        if rad_id not in annotations:
            annotations[rad_id] = {
                "boxes_list": [],
                "scores_list": [],
                "labels_list": [],
            }

            # We consider all of the radiologists as equal.
            weights.append(1.0)

        annotations[rad_id]["boxes_list"].append([row["x_min"], row["y_min"], row["x_max"], row["y_max"]])
        annotations[rad_id]["scores_list"].append(1.0)
        annotations[rad_id]["labels_list"].append(row["class_id"])

    boxes_list = []
    scores_list = []
    labels_list = []

    for annotator in annotations.keys():
        boxes_list.append(annotations[annotator]["boxes_list"])
        scores_list.append(annotations[annotator]["scores_list"])
        labels_list.append(annotations[annotator]["labels_list"])

    # Calculate WBF
    boxes, scores, labels = weighted_boxes_fusion(
        boxes_list,
        scores_list,
        labels_list,
        weights=weights,
        iou_thr=iou_thr,
        skip_box_thr=skip_box_thr
    )

    for idx, box in enumerate(boxes):
        class_name=row["class_name"]
        results.append({
            "image_id": image_id,
            "class_id": int(labels[idx]),
            "rad_id": "wbf",
            "x_min": box[0] * max_value,
            "y_min": box[1] * max_value,
            "x_max": box[2] * max_value,
            "y_max": box[3] * max_value,
            "width": width,
            "height": height,
            "image_path": image_path,
            "class_name": mapping[int(labels[idx])]
        })

results = pd.DataFrame(results)

  0%|          | 0/4394 [00:00<?, ?it/s]

In [ ]:
results.shape

(23904, 11)

In [ ]:
results.head()

,image_id,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path,class_name
0,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,690.666676,1354.333235,1658.666714,1797.666677,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Cardiomegaly
1,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,1052.000000,715.000000,1299.000000,966.000000,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Aortic enlargement
2,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,1789.000000,1729.000000,1875.000000,1992.000000,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Pleural thickening
3,9a5094b2563a1ef3ff50dc5c7ff71345,10,wbf,1789.000000,1729.000000,1875.000000,1992.000000,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Pleural effusion
4,051132a778e61a86eb147c7c6f564dfe,3,wbf,953.999977,1304.999961,2043.666679,1672.999894,2304,2880,/home/ubuntu/GitHub/Data/vinbigdata/train/0511...,Cardiomegaly


In [ ]:
train_df=results

Normalize data to between 0-1


In [ ]:
train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.head()

,image_id,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path,class_name,x_mid,y_mid,w,h,area
0,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,0.332051,0.579766,0.797436,0.769549,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Cardiomegaly,0.564744,0.674658,0.465385,0.189783,0.088322
1,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,0.505769,0.306079,0.624519,0.413527,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Aortic enlargement,0.565144,0.359803,0.118750,0.107449,0.012760
2,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,0.860096,0.740154,0.901442,0.852740,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Pleural thickening,0.880769,0.796447,0.041346,0.112586,0.004655
3,9a5094b2563a1ef3ff50dc5c7ff71345,10,wbf,0.860096,0.740154,0.901442,0.852740,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Pleural effusion,0.880769,0.796447,0.041346,0.112586,0.004655
4,051132a778e61a86eb147c7c6f564dfe,3,wbf,0.414062,0.453125,0.887008,0.580903,2304,2880,/home/ubuntu/GitHub/Data/vinbigdata/train/0511...,Cardiomegaly,0.650535,0.517014,0.472946,0.127778,0.060432


In [ ]:
features = ['x_min', 'y_min', 'x_max', 'y_max', 'x_mid', 'y_mid', 'w', 'h', 'area']
X = train_df[features]
y = train_df['class_id']
X.shape, y.shape

((23904, 9), (23904,))

# 5.Train-Val Split

GroupKFold

In [ ]:
fold=0
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups = train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
train_df.head()

,image_id,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path,class_name,x_mid,y_mid,w,h,area,fold
0,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,0.332051,0.579766,0.797436,0.769549,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Cardiomegaly,0.564744,0.674658,0.465385,0.189783,0.088322,2
1,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,0.505769,0.306079,0.624519,0.413527,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Aortic enlargement,0.565144,0.359803,0.118750,0.107449,0.012760,2
2,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,0.860096,0.740154,0.901442,0.852740,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Pleural thickening,0.880769,0.796447,0.041346,0.112586,0.004655,2
3,9a5094b2563a1ef3ff50dc5c7ff71345,10,wbf,0.860096,0.740154,0.901442,0.852740,2080,2336,/home/ubuntu/GitHub/Data/vinbigdata/train/9a50...,Pleural effusion,0.880769,0.796447,0.041346,0.112586,0.004655,2
4,051132a778e61a86eb147c7c6f564dfe,3,wbf,0.414062,0.453125,0.887008,0.580903,2304,2880,/home/ubuntu/GitHub/Data/vinbigdata/train/0511...,Cardiomegaly,0.650535,0.517014,0.472946,0.127778,0.060432,0


Train-val split

You need to run the model five times. One time for each fold - fold0, fold1, fold2, fold3 and fold4. 


In [ ]:
fold=0
train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
#train_files

After you have run the model for one fold you need to delete the following directories and manually delete the the other files, just keeping the vinbigdata folder and the train_merge file.

In [ ]:
len(train_files), len(val_files)

(3515, 879)

In [ ]:
#Create folders for train and validation and the corresponding labels and move them to the right location

os.makedirs('/home/ubuntu/GitHub/Data/vinbigdata/labels/train', exist_ok = True)
os.makedirs('/home/ubuntu/GitHub/Data/vinbigdata/labels/val', exist_ok = True)
os.makedirs('/home/ubuntu/GitHub/Data/vinbigdata/images/train', exist_ok = True)
os.makedirs('/home/ubuntu/GitHub/Data/vinbigdata/images/val', exist_ok = True)
label_dir = '/home/ubuntu/GitHub/Data/labels'
for file in tqdm(train_files):
  try:
    shutil.copy(file, '/home/ubuntu/GitHub/Data/vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/home/ubuntu/GitHub/Data/vinbigdata/labels/train')
  except:
    print("No such file")
    
for file in tqdm(val_files):
  try:
    shutil.copy(file, '/home/ubuntu/GitHub/Data/vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/home/ubuntu/GitHub/Data/vinbigdata/labels/val')
  except:
    print("No such file")

  0%|          | 0/3515 [00:00<?, ?it/s]

  0%|          | 0/879 [00:00<?, ?it/s]

Change the labels to correspond to WBF

In [ ]:
labels = os.listdir('/home/ubuntu/GitHub/Data/vinbigdata/labels/train')
cwd = "/home/ubuntu/GitHub/Data/vinbigdata/labels/train"
for label in tqdm(labels):
  filepath = os.path.join(cwd, label)
  img_id = str(label.split('.')[0])

  l = len(train_df[train_df.image_id == img_id])
  df = train_df[train_df.image_id == img_id]
  LST = []
  for index, row in df.iterrows():
     lst = []
     lst.append(row["class_id"])

     box_w=row["x_max"]-row["x_min"]
     box_h=row["y_max"]-row["y_min"]
     box_x=row["x_min"]+(box_w/2)
     box_y=row["y_min"]+(box_h/2)
     
     lst.append(box_x)
     lst.append(box_y)
     lst.append(box_w)
     lst.append(box_h)
     
     LST.append(' '.join(str(e) for e in lst))

  text_list = []
  for i in range(len(LST)):
    text = str(LST[i]) + str("\n")
    text_list.append(text)

  labels_file = open(filepath, "w")
  labels_file.writelines(text_list)
  labels_file.close()


  0%|          | 0/3515 [00:00<?, ?it/s]

In [ ]:
labels = os.listdir('/home/ubuntu/GitHub/Data/vinbigdata/labels/val')
cwd = "/home/ubuntu/GitHub/Data/vinbigdata/labels/val"
for label in tqdm(labels):
  filepath = os.path.join(cwd, label)
  img_id = str(label.split('.')[0])

  l = len(train_df[train_df.image_id == img_id])
  df = train_df[train_df.image_id == img_id]
  LST = []
  for index, row in df.iterrows():
     lst = []
     lst.append(row["class_id"])

     box_w=row["x_max"]-row["x_min"]
     box_h=row["y_max"]-row["y_min"]
     box_x=row["x_min"]+(box_w/2)
     box_y=row["y_min"]+(box_h/2)
     
     lst.append(box_x)
     lst.append(box_y)
     lst.append(box_w)
     lst.append(box_h)
     
     LST.append(' '.join(str(e) for e in lst))

  text_list = []
  for i in range(len(LST)):
    text = str(LST[i]) + str("\n")
    text_list.append(text)

  labels_file = open(filepath, "w")
  labels_file.writelines(text_list)
  labels_file.close()

  0%|          | 0/879 [00:00<?, ?it/s]

# 6.YOLO hyperparameters

The next two cells are two different set of hyperparameters. 
The best would be if you run all five folds with one set and then all five folds with the other set of hyperparameters. If this takes too long choose the first set only and run five folds.

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

#cwd = '/content/'
cwd = '/home/ubuntu/GitHub/'

# Hyperparameters
hyp = {'lr0': 0.01,  # initial learning rate (SGD=1E-2, Adam=1E-3)
       'lrf': 0.2,  # final OneCycleLR learning rate (lr0 * lrf)
       'momentum': 0.937,  # SGD momentum
       'weight_decay': 5e-4,  # optimizer weight decay
       'warmup_epochs': 3.0,  # warmup epochs (fractions ok)
       'warmup_momentum': 0.8,  # warmup initial momentum
       'warmup_bias_lr': 0.1,  # warmup initial bias lr
       'box': 0.05,  # box loss gain
       'giou': 0.05,  # giou loss gain
       'cls': 0.5,  # cls loss gain
       'cls_pw': 1.0,  # cls BCELoss positive_weight
       'obj': 1.0,  # obj loss gain (*=img_size/320 if img_size != 320)
       'obj_pw': 1.0,  # obj BCELoss positive_weight
       'iou_t': 0.20,  # iou training threshold (TRY LOWERING THIS)
       'anchor_t': 4.0,  # anchor-multiple threshold
       'anchors': 3,  # anchors per output layer (0 to ignore)
       'fl_gamma': 0,  # focal loss gamma (efficientDet default is gamma=1.5)
       'hsv_h': 0.014,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.68,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.36,  # image HSV-Value augmentation (fraction)
       'degrees': 0.0,  # image rotation (+/- deg)
       'translate': 0.0,  # image translation (+/- fraction)
       'scale': 0.5,  # image scale (+/- gain)
       'shear': 0.0,  # image shear (+/- deg)
       'perspective': 0.0,  # image perspective (+/- fraction), range 0-0.001
       'flipud': 0.0,  # image flip up-down (probability)
       'fliplr': 0.5,  # image flip left-right (probability)
       'mosaic': 1.0,  # image mosaic (probability)
       'mixup': 0.0}  # image mixup (probability)


with open(join( cwd , 'hyp_new.yaml'), 'w') as outfile:
    yaml.dump(hyp, outfile, default_flow_style=False)

f = open(join( cwd , 'hyp_new.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
anchor_t: 4.0
anchors: 3
box: 0.05
cls: 0.5
cls_pw: 1.0
degrees: 0.0
fl_gamma: 0
fliplr: 0.5
flipud: 0.0
giou: 0.05
hsv_h: 0.014
hsv_s: 0.68
hsv_v: 0.36
iou_t: 0.2
lr0: 0.01
lrf: 0.2
mixup: 0.0
momentum: 0.937
mosaic: 1.0
obj: 1.0
obj_pw: 1.0
perspective: 0.0
scale: 0.5
shear: 0.0
translate: 0.0
warmup_bias_lr: 0.1
warmup_epochs: 3.0
warmup_momentum: 0.8
weight_decay: 0.0005



Use these hyperparamters.
They are from this thread and seems to improve the results.
But only in the case we set img size to 640 when we train. In case you are not using data augmentation set "flipud": 0.2 and
"fliplr": 0.5.

[Discussion](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/222707)


# 7.YOLO yaml-file

Here we specify the classes and provide locations to text-files with the train, val and test image names.

Get Class Name

In [ ]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes=list(set(classes))

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = '/home/ubuntu/GitHub/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/home/ubuntu/GitHub/Data/vinbigdata/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/home/ubuntu/GitHub/Data/vinbigdata/images/val/*'):
        f.write(path+'\n')

with open(join( cwd , 'test.txt'), 'w') as f:
    for path in glob('/home/ubuntu/GitHub/Data/vinbigdata/test/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    test =  join( cwd , 'test.txt'),
    nc    = 14,
    names = classes
    )

with open(join( cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- Consolidation
- Infiltration
- ILD
- Calcification
- Lung Opacity
- Nodule/Mass
- Pleural effusion
- Pneumothorax
- Atelectasis
- Cardiomegaly
- Pulmonary fibrosis
- Aortic enlargement
- Pleural thickening
- Other lesion
nc: 14
test: /home/ubuntu/GitHub/test.txt
train: /home/ubuntu/GitHub/train.txt
val: /home/ubuntu/GitHub/val.txt



# 8.Training

In [ ]:
os.chdir('/home/ubuntu/GitHub/yolov5')
import torch

In [ ]:
#It has been some problems with this the last day. You need to click, then wait and let it restart and then click again to use it.
#Or use !WANDB_MODE="dryrun"
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

Uncomment the one you want to run:

50 epochs seems enough for the model to converge.
16 batch size seems good.
Check out this link for an investigation of optimal batch size:
[Batch size](https://github.com/ultralytics/yolov5/issues/2377)

In [ ]:
# https://stackoverflow.com/questions/42984743/nvidia-smi-has-failed-because-it-couldnt-communicate-with-the-nvidia-driver
# Check that a GPU is running
!nvidia-smi

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Sat Mar 20 23:54:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   52C    P8    29W / 149W |      3MiB / 11441MiB |      0%    

In [ ]:
#!WANDB_MODE="dryrun" python train.py --img 1280 --batch 4 --epochs 50 --data /content/vinbigdata.yaml --cache --cfg ./models/yolov5x.yaml --weights yolov5x.pt --device 0 --hyp /content/hyp_new.yaml 


!python train.py --img 1280 --batch 2 --epochs 50 --data /home/ubuntu/GitHub/vinbigdata.yaml --cache --cfg ./models/yolov5x.yaml --weights yolov5x.pt --hyp /home/ubuntu/GitHub/hyp_new.yaml


# ERRORS:
# If you get a semaphore error, just try to run this cell again, that usually solves it.


/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v4.0-138-ged2c742 torch 1.8.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, batch_size=2, bucket='', cache_images=True, cfg='./models/yolov5x.yaml', data='/home/ubuntu/GitHub/vinbigdata.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='/home/ubuntu/GitHub/hyp_new.yaml', image_weights=False, img_size=[1280, 1280], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp4', single_cls=False, sync_bn=False, total_batch_size=2, weights

# 14.Inference

Make sure you use the right exp here. You can see which experiment you are running during the training. (Check the folder and use the latest exp. Its exp for experiment 1, then exp2 for experiment 2 etc.)

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.001 --iou 0.4 --source /content/vinbigdata/test --save-txt --save-conf --exist-ok

# 15.Prediction and submission

In [ ]:
# credit / source https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
# credit / source https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer
image_ids = []
PredictionStrings = []
test_df = pd.read_csv('/content/vinbigdata/test.csv')

for file_path in tqdm(glob('/content/yolov5/runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0] # extract image id
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

In [ ]:
# credit / source: https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df.to_csv('/content/1280new_foldx_no2class.csv',index = False)
sub_df.tail()